In [ ]:
# -*- coding: utf-8 -*-
"""5_LDA_v3.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1rS4ARb_aioj639IwqB6BWI8sqnENeoFD
"""

import io
import os.path
import re
import tarfile

import smart_open

def extract_documents(url='https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'):
    fname = url.split('/')[-1]

    # Download the file to local storage first.
    # We can't read it on the fly because of
    # https://github.com/RaRe-Technologies/smart_open/issues/331
    if not os.path.isfile(fname):
        with smart_open.open(url, "rb") as fin:
            with smart_open.open(fname, 'wb') as fout:
                while True:
                    buf = fin.read(io.DEFAULT_BUFFER_SIZE)
                    if not buf:
                        break
                    fout.write(buf)

    with tarfile.open(fname, mode='r:gz') as tar:
        # Ignore directory entries, as well as files like README, etc.
        files = [
            m for m in tar.getmembers()
            if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
        ]
        for member in sorted(files, key=lambda x: x.name):
            member_bytes = tar.extractfile(member).read()
            yield member_bytes.decode('utf-8', errors='replace')











In [ ]:
import pandas as pd
docs = pd.read_csv('/content/books_data (1) (1).csv')

print(len(docs))
print(docs[:200])


1000
     Unnamed: 0 Author                                               Text
0             0      a  an invitation from Hartfield , and who were fe...
1             1      a  materially changed since we talked on this sub...
2             2      a  , gentlemanlike man , whose frequent visits we...
3             3      a  me , whenever I had an opportunity , to thank ...
4             4      a  such unfortunate circumstances do sometimes oc...
..          ...    ...                                                ...
195         195      a  a few steps nearer , and those few steps were ...
196         196      a  days of his favour , none had disturbed her mo...
197         197      a  ever look at him in the company of the other ....
198         198      a  the children -- a most mortifying change , and...
199         199      a  my side . He has imposed on me , but he has no...

[200 rows x 3 columns]


In [ ]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs.iloc[idx]['Text'] = docs.iloc[idx]['Text'].lower()  # Convert to lowercase.
    docs.iloc[idx]['Text'] = tokenizer.tokenize(docs.iloc[idx]['Text'])  # Split into words.
i=0
for doc in docs['Text']:
  a=[]
  for token in doc:
    if (not token.isnumeric()) and len(token)>1:
      a.append(token)
  docs.iloc[i]['Text']=a
  i+=1
# # Remove numbers, but not words that contain numbers.
# docs['Text'] = [[token for token in doc if not token.isnumeric()] for doc in docs['Text']]

# # Remove words that are only one character.
# docs['Text'] = [[token for token in doc if len(token) > 1] for doc in docs['Text']]


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [ ]:
docs

,Unnamed: 0,Author,Text
0,0,a,"an invitation from Hartfield , and who were fe..."
1,1,a,materially changed since we talked on this sub...
2,2,a,", gentlemanlike man , whose frequent visits we..."
3,3,a,"me , whenever I had an opportunity , to thank ..."
4,4,a,such unfortunate circumstances do sometimes oc...
...,...,...,...
995,995,e,"sleeps his meadow , and there sleep his cattle..."
996,996,e,the stump ; leaning on a shivered lance ; prop...
997,997,e,"d ' ye see ; now as you well know , sir , we c..."
998,998,e,there again ! there she breaches ! right ahead...


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
# Importing necessary libraries
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatiser = WordNetLemmatizer()
# Defining a module for Text Processing
def text_process(tex):
    # 1. Removal of Punctuation Marks 
    nopunct=[char for char in tex if char not in string.punctuation]
    nopunct=''.join(nopunct)
    # 2. Lemmatisation 
    a=''
    i=0
    for i in range(len(nopunct.split())):
        b=lemmatiser.lemmatize(nopunct.split()[i], pos="v")
        a=a+b+' '
    # 3. Removal of Stopwords
    return [word for word in a.split() if word.lower() not in stopwords.words('english')]


# import nltk
# nltk.download('wordnet')
# # Lemmatize the documents.
# from nltk.stem.wordnet import WordNetLemmatizer

# lemmatizer = WordNetLemmatizer()
# docs['Text'] = [[lemmatizer.lemmatize(t) for t in doc]for doc in docs['Text']]
docs['Text'] = docs['Text'].apply(text_process)
docs.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,Unnamed: 0,Author,Text
0,0,a,"[invitation, Hartfield, fetch, carry, home, of..."
1,1,a,"[materially, change, since, talk, subject, hop..."
2,2,a,"[gentlemanlike, man, whose, frequent, visit, o..."
3,3,a,"[whenever, opportunity, thank, could, thank, m..."
4,4,a,"[unfortunate, circumstances, sometimes, occur,..."


In [ ]:
docs

,Unnamed: 0,Author,Text
0,0,a,"[invitation, Hartfield, fetch, carry, home, of..."
1,1,a,"[materially, change, since, talk, subject, hop..."
2,2,a,"[gentlemanlike, man, whose, frequent, visit, o..."
3,3,a,"[whenever, opportunity, thank, could, thank, m..."
4,4,a,"[unfortunate, circumstances, sometimes, occur,..."
...,...,...,...
995,995,e,"[sleep, meadow, sleep, cattle, yonder, cottage..."
996,996,e,"[stump, lean, shiver, lance, prop, lonely, foo..."
997,997,e,"[ye, see, well, know, sir, claim, discharge, s..."
998,998,e,"[breach, right, ahead, White, Whale, White, Wh..."


In [ ]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs['Text'], min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs.iloc[idx]['Text']]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs.iloc[idx]['Text'].append(token)


2022-03-03 12:31:06,582 : INFO : collecting all words and their counts
2022-03-03 12:31:06,584 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-03-03 12:31:06,742 : INFO : collected 44850 word types from a corpus of 60212 words (unigram + bigrams) and 1000 sentences
2022-03-03 12:31:06,754 : INFO : using 44850 counts as vocab in Phrases<0 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>
/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
docs

,Unnamed: 0,Author,Text
0,0,a,"[invitation, Hartfield, fetch, carry, home, of..."
1,1,a,"[materially, change, since, talk, subject, hop..."
2,2,a,"[gentlemanlike, man, whose, frequent, visit, o..."
3,3,a,"[whenever, opportunity, thank, could, thank, m..."
4,4,a,"[unfortunate, circumstances, sometimes, occur,..."
...,...,...,...
995,995,e,"[sleep, meadow, sleep, cattle, yonder, cottage..."
996,996,e,"[stump, lean, shiver, lance, prop, lonely, foo..."
997,997,e,"[ye, see, well, know, sir, claim, discharge, s..."
998,998,e,"[breach, right, ahead, White, Whale, White, Wh..."


In [ ]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
#dictionary = Dictionary(docs['Text'][400:600]) #2nd
dictionary = Dictionary(docs['Text'][800:1000]) #4th
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

# Bag-of-words representation of the documents.
#corpus = [dictionary.doc2bow(doc) for doc in docs['Text'][400:600]]
corpus = [dictionary.doc2bow(doc) for doc in docs['Text'][800:1000]]


print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

2022-03-03 12:35:22,339 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-03-03 12:35:22,365 : INFO : built Dictionary(4372 unique tokens: ['Atlantic', 'Ho', 'Town', 'account', 'across']...) from 200 documents (total 12614 corpus positions)
2022-03-03 12:35:22,378 : INFO : discarding 4314 tokens: [('Atlantic', 3), ('Ho', 5), ('Town', 4), ('account', 8), ('across', 6), ('altogether', 5), ('bottom', 12), ('chorus', 2), ('civilize', 5), ('clang', 1)]...
2022-03-03 12:35:22,380 : INFO : keeping 58 tokens which were in no less than 20 and no more than 100 (=50.0%) documents
2022-03-03 12:35:22,385 : INFO : resulting dictionary: Dictionary(58 unique tokens: ['day', 'even', 'every', 'hand', 'know']...)


Number of unique tokens: 58
Number of documents: 200


In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

top_topics = model.top_topics(corpus) #, num_words=20)

2022-03-03 12:35:25,442 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2022-03-03 12:35:25,446 : INFO : using serial LDA version on this node
2022-03-03 12:35:25,450 : INFO : running online (multi-pass) LDA training, 10 topics, 20 passes over the supplied corpus of 200 documents, updating model once every 200 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-03-03 12:35:25,460 : INFO : PROGRESS: pass 0, at document #200/200
2022-03-03 12:35:25,625 : INFO : optimized alpha [0.07371848, 0.07491694, 0.08453762, 0.08641265, 0.074944586, 0.0794211, 0.0731354, 0.08018313, 0.0951083, 0.072436705]
2022-03-03 12:35:25,628 : INFO : topic #9 (0.072): 0.080*"man" + 0.061*"ye" + 0.051*"whale" + 0.051*"say" + 0.041*"every" + 0.041*"day" + 0.031*"head" + 0.031*"see" + 0.031*"eye" + 0.031*"look"
2022-03-03 12:35:25,633 : INFO : topic #6 (0.073): 0.049*"upon" + 0.049*"one" + 0.049*"hand" + 0

In [ ]:
# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)





Average topic coherence: -1.8452.
[([(0.14544728, 'old'),
   (0.097410105, 'Ahab'),
   (0.055874653, 'cry'),
   (0.051142268, 'like'),
   (0.0467555, 'man'),
   (0.030727101, 'one'),
   (0.030222747, 'yet'),
   (0.02836643, 'sail'),
   (0.024798868, 'water'),
   (0.024073, 'must'),
   (0.022434305, 'look'),
   (0.021639992, 'long'),
   (0.020156087, 'round'),
   (0.019062115, 'eye'),
   (0.018377008, 'two'),
   (0.017969675, 'head'),
   (0.017801141, 'ship'),
   (0.017183831, 'go'),
   (0.0167221, 'seem'),
   (0.014875313, 'whale')],
  -1.6146325428791446),
 ([(0.09074696, 'one'),
   (0.08637735, 'life'),
   (0.056764048, 'little'),
   (0.048194453, 'boat'),
   (0.043778602, 'time'),
   (0.042854983, 'Ahab'),
   (0.04180588, 'last'),
   (0.032522142, 'hand'),
   (0.03251475, 'seem'),
   (0.029569335, 'take'),
   (0.029246405, 'make'),
   (0.02883094, 'whale'),
   (0.028013118, 'Whale'),
   (0.023007523, 'men'),
   (0.02250062, 'like'),
   (0.022063948, 'man'),
   (0.021127863, 'even'),

In [ ]:
!pip install pyLDAvis

In [ ]:
# import pyLDAvis.gensim
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(model, corpus, dictionary=dictionary)
# vis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(model, corpus, dictionary)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.051923 -0.017950       1        1  18.538627
2      0.063609 -0.083860       2        1  14.006728
3     -0.048374 -0.056363       3        1  13.130695
7      0.043454  0.110797       4        1  10.322879
5      0.127170 -0.022168       5        1   9.097265
1      0.003568 -0.042695       6        1   8.983105
6     -0.093050 -0.031809       7        1   8.431261
0     -0.069112  0.046605       8        1   6.196214
9      0.065664  0.038337       9        1   5.853900
4     -0.041005  0.059107      10        1   5.439327, topic_info=     Term        Freq       Total Category  logprob  loglift
24     ye   68.000000   68.000000  Default  30.0000  30.0000
50    old   54.000000   54.000000  Default  29.0000  29.0000
29    one   89.000000   89.000000  Default  28.0000  28.0000
20  whale  122.000000  122.000000  Default  27.0000  27.0000
30    say   85.000000   85.000000  Default  26.0000  26.0000
..    ...         ...         ...      ...      ...      ...
51  round    1.233850   29.990019  Topic10  -4.6498  -0.2792
26     go    1.948358   53.071929  Topic10  -4.1930  -0.3931
5    like    2.903049   84.752036  Topic10  -3.7942  -0.4625
20  whale    3.719792  122.518878  Topic10  -3.5463  -0.5831
30    say    2.419452   85.783865  Topic10  -3.9764  -0.6568

[366 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
32        1  0.019299  Ahab
32        2  0.096493  Ahab
32        3  0.077195  Ahab
32        4  0.463167  Ahab
32        5  0.038597  Ahab
...     ...       ...   ...
24        9  0.029312    ye
21        1  0.282575   yet
21        2  0.188384   yet
21        4  0.329671   yet
21        7  0.094192   yet

[497 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 3, 4, 8, 6, 2, 7, 1, 10, 5])